# ERA5-Land Monthly Averaged

* [ERA5-Land Monthly Averaged](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview)
* [Documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation)

## Import dependencies

In [49]:
import os
import cdsapi
import ipywidgets as widgets
from datetime import datetime

from cds_client_util import initialize_cds_client

## Manage Working Directories

In [50]:
# The directories (if doesn't exist) will be created in the Data folder
download_folder = r".\data\era5-land-monthly-means\download"
working_folder = r".\data\era5-land-monthly-means\working"
output_folder = r".\data\era5-land-monthly-means\output"

if not os.path.exists(working_folder):
    os.makedirs(working_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

## CDS API key and authentication

In [51]:
# Use the function to initialize the client
try:
    client = initialize_cds_client("../secrets/.cdsapirc")
    print("CDS API Client initialized successfully!")
except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")

In [52]:
# Alternative: Enter your CDS API key directly
# ------------------------------------------------------
# If you have not created the ".cdsapirc" file or prefer not to use it, 
# you can manually input your CDS API key in this notebook cell.
# This approach is helpful for quick testing or one-off uses but is not recommended 
# for long-term or automated workflows as it requires manual input and may expose your key.
# ------------------------------------------------------

# import getpass

# api_key = getpass.getpass("Enter your CDS API Key:")
# api_url = "https://cds.climate.copernicus.eu/api"
# client = cdsapi.Client(url=api_url, key=api_key)

## Create list of variables

The list of variables are available at [ERA5-Land Monthly Averaged](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview)

*Please check and modify the predefined variable name if any issue arises with variable name.* 

In [53]:
# Variable Groups
var_group_temperature = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4"]

var_group_lake = [
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    "lake_total_layer_temperature"]

var_group_snow = [
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer"]

var_group_soil_water = [
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4"]

var_group_radiation_and_heat = [
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards"]

var_group_evaporation_and_runoff = [
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation"]

var_group_wind_pressure_and_precipitation = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation"]

var_group_vegetation = [
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"]

# List of variable group
var_group_list = ['var_group_temperature',
                  'var_group_lake',
                  'var_group_snow',
                  'var_group_soil_water',
                  'var_group_radiation_and_heat',
                  'var_group_evaporation_and_runoff',
                  'var_group_wind_pressure_and_precipitation',
                  'var_group_vegetation']

## Define request parameters for download

In [54]:
selected_variable_group = widgets.Dropdown(
    options=var_group_list,
    value=var_group_list[0],
    description="Select the variable group",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable_group

In [55]:
current_variable_group = globals().get(selected_variable_group.value)

selected_variable = widgets.Dropdown(
    options=current_variable_group,
    value=current_variable_group[1],
    description="Select the variable of interest",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable

In [56]:
# Define the bounding box extents in CRS: WGS84 in this format [lat_max, lon_min, lat_min, lon_max]
# Recommended precision is 0.1 because Horizontal resolution: 0.1° x 0.1°; Native resolution is 9 km.

bbox_wgs84_deutschland = [56.0, 5.8, 47.2, 15.0] # Bouding box for Germany
bbox_wgs84_konstanz = [47.8, 9, 47.6, 9.3] # Bounding box for Konstanz, Germany

In [57]:
selected_year = widgets.Dropdown(
    options=[str(year) for year in range(1950, datetime.now().year + 1)],
    value=str(datetime.now().year),
    description="Select the year for downloading data:",
    disabled=False,
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_year

## Define request parameters and download the dataset

In [ ]:
# This request downloads 1 year data: 12 months (365 days, 366 days for leap years) and monthly averaged at 00:00.
# The total length of the downloaded dataset is 12*1=12

dataset = "reanalysis-era5-land-monthly-means"
dataset_filename = f"{dataset}_{selected_variable.value}_{selected_year.value}.nc"
dataset_filepath = os.path.join(download_folder,dataset_filename)

request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": selected_variable.value,
    "year": selected_year.value,
    "month": [str(month) for month in range(13)],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": bbox_wgs84_konstanz
}


# Download the dataset with the defined request paremeters
client.retrieve(dataset, request, dataset_filepath)

## Example requests for use case

In [ ]:
# This request downloads year 1950 to 2024 data, monthly averaged at 00:00
# "variable": ["2m_temperature"],

# dataset = "reanalysis-era5-land-monthly-means"
# dataset_filename = f"{dataset}_2m_temperature_1950_2024.nc"
# dataset_filepath = os.path.join(download_folder,dataset_filename)

# request = {
#     "product_type": ["monthly_averaged_reanalysis"],
#     "variable": ["2m_temperature"],
#     "year": [str(year) for year in range(1950,2024+1,1)],
#     "month": [str(month) for month in range(13)],
#     "time": ["00:00"],
#     "data_format": "netcdf",
#     "download_format": "unarchived",
#     "area": bbox_wgs84_konstanz
# }

# # Download the dataset with the defined request paremeters
# client.retrieve(dataset, request, dataset_filepath)

In [ ]:
# This request downloads year 1950 to 2024 data, monthly averaged at 00:00

# dataset = "reanalysis-era5-land-monthly-means"
# dataset_filename = f"{dataset}_snow_cover_1950_2024.nc"
# dataset_filepath = os.path.join(download_folder,dataset_filename)

# request = {
#     "product_type": ["monthly_averaged_reanalysis"],
#     "variable": ["snow_cover"],
#     "year": [str(year) for year in range(1950,2024+1,1)],
#     "month": [str(month) for month in range(1,4,1)],
#     "time": ["00:00"],
#     "data_format": "netcdf",
#     "download_format": "unarchived",
#     "area": bbox_wgs84_konstanz
# }

# # Download the dataset with the defined request paremeters
# client.retrieve(dataset, request, dataset_filepath)

In [ ]:
# This request downloads year 1950 to 2024 data, monthly averaged at 00:00

dataset = "reanalysis-era5-land-monthly-means"
dataset_filename = f"{dataset}_lake_ice_depth_1950_2024.nc"
dataset_filepath = os.path.join(download_folder,dataset_filename)

request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": ["lake_ice_depth"],
    "year": [str(year) for year in range(1950,2024+1,1)],
    "month": ["01","02","03","11","12"],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": bbox_wgs84_konstanz
}

# Download the dataset with the defined request paremeters
client.retrieve(dataset, request, dataset_filepath)

2024-11-21 17:09:09,106 INFO Request ID is 477cb92b-dfee-4964-b630-36f6a3dba143
2024-11-21 17:09:09,171 INFO status has been updated to accepted
2024-11-21 17:09:11,805 INFO status has been updated to running
2024-11-21 17:11:03,481 INFO status has been updated to failed


HTTPError: 400 Client Error: Bad Request for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/477cb92b-dfee-4964-b630-36f6a3dba143/results
The job has failed.
